# Albumentations

In [ ]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# # unmount 필요한 경우
# drive.flush_and_unmount()


In [ ]:
# 패키지 설치 - model 관련 & albumentations 관련
# 설치후 restart warning 나오면 런타임 다시 시작하기

!pip install ultralytics
!pip install roboflow
!pip install pybboxes

# 파이썬에서 YAML 파일을 사용하기 위해서 PyYAML 라이브러리 설치
!pip install PyYaml

# albumentations 설치
# !pip install -r requirements.txt    # albumentations 1.3.0 깔기
# !pip install albumentations==1.3.0
# !pip install --upgrade albumentations
!pip install -U git+https://github.com/albumentations-team/albumentations     # 최신 버전 깔기
# !pip install --upgrade albumentations


In [ ]:
# web git clone
!git clone https://github.com/likespike/hmkd1_2nd_project_semiconductor_defect_detection.git


In [ ]:
# web git clone일 경우에만 해당
# got clone 내용 -> content 폴더로 이동시키기
# model 폴더를 제외한 모든 폴더 및 파일이 import를 위해 content 디렉터리에 위치해야 함

import os
import shutil

source_folder = '/content/hmkd1_2nd_project_semiconductor_defect_detection'
destination_folder = '/content/'

# 대상 폴더에 중복된 이름의 파일이 있는지 확인하고, 필요한 경우 파일을 복사하여 중복을 피합니다.
for item in os.listdir(source_folder):
    source_path = os.path.join(source_folder, item)
    destination_path = os.path.join(destination_folder, item)

    # 대상 폴더에 중복된 이름의 파일이 있는지 확인하고, 중복된 이름이라면 파일을 복사하여 다른 이름으로 저장합니다.
    if os.path.exists(destination_path):
        base_name, ext = os.path.splitext(item)
        count = 1
        while os.path.exists(os.path.join(destination_folder, f"{base_name}_{count}{ext}")):
            count += 1
        destination_path = os.path.join(destination_folder, f"{base_name}_{count}{ext}")

    # 폴더와 파일을 이동합니다.
    shutil.move(source_path, destination_path)

# 원본 폴더를 삭제합니다.
shutil.rmtree(source_folder)


In [ ]:
# # roboflow dataset을 받아 할 경우

# from roboflow import Roboflow
# rf = Roboflow(api_key="R7Sv3XZ9Wk5orzpjlsLw")
# project = rf.workspace("data-universe").project("semiconductor-defect-detection")
# dataset = project.version(66).download("yolov8")

# # 앞으로의 경로 설정에 쓸 dataset 경로 확인하기
# dataset.location


In [ ]:
# contants.yame 파일 내용 수정 (colab, jupyter notebook에서 파일 수정하고 싶을 경우)
import yaml
import os

# # roboflow dataset종류에 따른 자동 경로
# os.makedirs(os.path.join(dataset.location, 'augmentations/images'), exist_ok=True)
# os.makedirs(os.path.join(dataset.location, 'augmentations/labels'), exist_ok=True)

# data = {
#     'inp_img_pth' : os.path.join(dataset.location, 'train1/images'),
#     'inp_lab_pth' : os.path.join(dataset.location, 'train1/labels'),
#     'out_img_pth' : os.path.join(dataset.location, 'augmentations/images'),
#     'out_lab_pth' : os.path.join(dataset.location, 'augmentations/labels'),
#     'transformed_file_name' : 'aug_out',

#     #  클래스는 연속성과 개수만 맞으면 되는듯함
#     # 'CLASSES' : ['obj1', 'obj2', 'obj3', 'obj4', 'obj5', 'obj6', 'obj7', 'obj8', 'obj9', 'obj10', 'obj11', 'obj12'],
#     # 'CLASSES' : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
#     'CLASSES' : ['broken_line', 'circle', 'diagonal_line', 'dot', 'out_line', 'quarter_line', 'scratch', 'semicircle', 'short_line', 'square_line', 'thick_line', 'thin_line']
# }

# runtime 재시작 등으로 roboflow dataset 반복 다운 귀찮을 때 수동 경로 지정
# os.makedirs('/content/augmentations/images', exist_ok=True)
# os.makedirs('/content/augmentations/labels', exist_ok=True)

data = { 'inp_img_pth' : '/content/drive/MyDrive/2nd_Project/Vision_Data/train_ori/images',
         'inp_lab_pth' : '/content/drive/MyDrive/2nd_Project/Vision_Data/train_ori/labels',
         'out_img_pth' : '/content/drive/MyDrive/2nd_Project/Vision_Data/temp/images',
         'out_lab_pth' : '/content/drive/MyDrive/2nd_Project/Vision_Data/temp/labels',
        #  'transformed_file_name' : 'aug_out',
         'CLASSES' : ['broken_line', 'circle', 'diagonal_line', 'dot', 'out_line', 'quarter_circle', 'scratch', 'semicircle', 'short_line', 'square_line', 'thick_line', 'thin_line'],
         'pre' : 'rotate90_',    # 주피터 노트북에서는 pipeline 실행할 때 pre를 그때그때마다 지정해 주는 것이 편함
         }

with open('/content/contants.yaml', 'w') as f:
    yaml.dump(data, f)

# data.yaml 읽어서 화면에 출력
with open('/content/contants.yaml', 'r') as f:
    data_yaml = yaml.safe_load(f)
    display(data_yaml)


In [ ]:
# Augmentations 옵션 수정 (colab, jupyter notebook에서 파일 수정하고 싶을 경우)
file_name = "/content/controller/apply_album_aug.py"

# 수정한 내용을 저장합니다.
new_content = """
from controller.album_to_yolo_bb import multi_obj_bb_yolo_conversion
from controller.album_to_yolo_bb import single_obj_bb_yolo_conversion
from controller.save_augs import save_aug_image, save_aug_lab
# from controller.validate_results import draw_yolo
import albumentations as A


def apply_aug(image, bboxes, out_lab_pth, out_img_pth, transformed_file_name, classes):
    transform = A.Compose([
        # A.RandomCrop(width=300, height=300),
        A.HorizontalFlip(always_apply=True, p=1.0)
        # A.RandomBrightnessContrast(p=-1),
        # A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0),
        # A.CLAHE(clip_limit=(0, 1), tile_grid_size=(8, 8), always_apply=True),
        # A.Resize(300, 300)
    ], bbox_params=A.BboxParams(format='yolo'))
    transformed = transform(image=image, bboxes=bboxes)
    transformed_image = transformed['image']
    transformed_bboxes = transformed['bboxes']
    tot_objs = len(bboxes)
    if tot_objs != 0:
        if tot_objs > 1:
            transformed_bboxes = multi_obj_bb_yolo_conversion(transformed_bboxes, classes)
            save_aug_lab(transformed_bboxes, out_lab_pth, transformed_file_name + ".txt")
        else:
            transformed_bboxes = [single_obj_bb_yolo_conversion(transformed_bboxes[0], classes)]
            save_aug_lab(transformed_bboxes, out_lab_pth, transformed_file_name + ".txt")
        save_aug_image(transformed_image, out_img_pth, transformed_file_name + ".png")
        # draw_yolo(transformed_image, transformed_bboxes)  # 작업 속도와 코랩 문제로 일단 안 씀
    else:
        # print("label file is empty")
        pass    # 추가함
"""
with open(file_name, "w") as file:
    file.write(new_content)


In [ ]:
# py파일 수정 제대로 됐는지 확인용
file_name = "/content/controller/apply_album_aug.py"

# 파일을 읽어서 수정합니다.
with open(file_name, "r") as file:
    content = file.read()

# 수정된 내용을 출력합니다.
print(content)


In [ ]:
# 실행
!python3 run.py


In [ ]:
# images, labels 폴더의 파일 개수가 일치하는지 확인하는 코드
import os

# directory = (os.path.join(dataset.location, 'augmentations/images'))
# directory1 = (os.path.join(dataset.location, 'augmentations/labels'))
directory = '/content/drive/MyDrive/2nd_Project/Vision_Data/temp/images'
directory1 = '/content/drive/MyDrive/2nd_Project/Vision_Data/temp/labels'

file_list = os.listdir(directory)
file_count = len(file_list)

file_list1 = os.listdir(directory1)
file_count1 = len(file_list1)

print("파일 개수:", file_count)
print("파일 개수:", file_count1)


In [ ]:
# 변환한 labels 정보들이 공백 두 칸이어서 'could not convert string to flot' 에러가 남
# 공백을 한 칸으로 바꿔 주는 코드
import os

# folder_path = os.path.join(dataset.location, 'augmentations/labels')  # roboflow 경로용
folder_path = '/content/drive/MyDrive/2nd_Project/Vision_Data/temp/labels'

def fix_spacing_in_files(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Fix the spacing in each line
    fixed_lines = [line.replace('  ', ' ') for line in lines]

    with open(file_path, 'w') as f:
        f.writelines(fixed_lines)

# Loop through each txt file in the folder and fix spacing
for file_name in os.listdir(folder_path):
    if file_name.endswith('.txt'):
        file_path = os.path.join(folder_path, file_name)
        fix_spacing_in_files(file_path)
        print(f"Fixed spacing in {file_name}")


In [ ]:
# images와 labels 파일 개수가 안 맞을 경우 images에만 있는 파일을 파악하는 코드
import os

image_folder = '/content/drive/MyDrive/2nd_Project/Vision_Data/temp/images'
label_folder = '/content/drive/MyDrive/2nd_Project/Vision_Data/temp/labels'

image_files = os.listdir(image_folder)
label_files = os.listdir(label_folder)

# 이미지 폴더에만 있는 파일 리스트를 찾습니다.
only_image_files = [file for file in image_files if file.split('.')[0] not in [label.split('.')[0] for label in label_files]]

# 이미지 폴더에만 있는 파일 개수를 출력합니다.
print("이미지 폴더에만 있는 파일 개수:", len(only_image_files))

# 이미지 폴더에만 있는 파일 리스트를 출력합니다.
print("이미지 폴더에만 있는 파일 리스트:")
for file in only_image_files:
    print(file)


In [ ]:
# images와 labels 파일 개수가 안 맞을 경우 images에만 있는 파일을 지워주는 코드
import os

# image_folder = os.path.join(dataset.location, 'augmentations/images')
# label_folder = os.path.join(dataset.location, 'augmentations/labels')
image_folder = '/content/drive/MyDrive/2nd_Project/Vision_Data/temp/images'
label_folder = '/content/drive/MyDrive/2nd_Project/Vision_Data/temp/labels'

image_files = os.listdir(image_folder)
label_files = os.listdir(label_folder)

# 이미지 폴더에만 있는 파일 리스트를 찾습니다.
only_image_files = [file for file in image_files if file.split('.')[0] not in [label.split('.')[0] for label in label_files]]

# 이미지 폴더에서 파일 삭제
for file in only_image_files:
    file_path = os.path.join(image_folder, file)
    os.remove(file_path)
    print(f"{file} 삭제 완료")

print("이미지 폴더에서 파일 삭제가 완료되었습니다.")


In [ ]:
# images와 labels 파일 개수가 안 맞을 경우 labels에만 있는 파일을 파악하는 코드
import os

images_folder_path = '/content/drive/MyDrive/2nd_Project/Vision_Data/train_accumulation1/images'
labels_folder_path = '/content/drive/MyDrive/2nd_Project/Vision_Data/train_accumulation1/labels'

# images 폴더 내 파일 이름 (확장자 제외) 추출
image_files = [os.path.splitext(f)[0] for f in os.listdir(images_folder_path) if os.path.isfile(os.path.join(images_folder_path, f))]

# labels 폴더 내 파일 이름 (확장자 제외) 추출
label_files = [os.path.splitext(f)[0] for f in os.listdir(labels_folder_path) if os.path.isfile(os.path.join(labels_folder_path, f))]

# labels 폴더에만 있는 파일 개수 세기
only_in_labels = [label for label in label_files if label not in image_files]
num_only_in_labels = len(only_in_labels)

print(f"labels 폴더에만 있는 파일 개수: {num_only_in_labels}개")


In [ ]:
# images와 labels 파일 개수가 안 맞을 경우 labels에만 있는 파일을 지워주는 코드
import os

images_folder_path = '/content/drive/MyDrive/2nd_Project/Vision_Data/train_shear/images'
labels_folder_path = '/content/drive/MyDrive/2nd_Project/Vision_Data/train_shear/labels'

# images 폴더 내 파일 이름 (확장자 제외) 추출
image_files = [os.path.splitext(f)[0] for f in os.listdir(images_folder_path) if os.path.isfile(os.path.join(images_folder_path, f))]

# labels 폴더 내 파일 이름 (확장자 제외) 추출
label_files = [os.path.splitext(f)[0] for f in os.listdir(labels_folder_path) if os.path.isfile(os.path.join(labels_folder_path, f))]

# labels 폴더에만 있는 파일 찾기
only_in_labels = [label for label in label_files if label not in image_files]

# labels 폴더에만 있는 파일 삭제
for file_name in only_in_labels:
    file_path = os.path.join(labels_folder_path, file_name + '.txt')  # .txt 확장자 추가
    if os.path.isfile(file_path):
        os.remove(file_path)
        print(f"파일 {file_name}.txt 삭제 완료")

print("작업 완료")


In [ ]:
# 필요한 경우 압축하여 local에 저장하는 코드
import os
import shutil
from google.colab import files

file_name = 'normal_h'

# 기존 zip 파일 삭제
existing_image_zip = '/content/'+file_name+'zip'
if os.path.exists(existing_image_zip):
    os.remove(existing_image_zip)

# 폴더 압축
shutil.make_archive("/content/"+file_name, 'zip', "/content/augmentations")

# 압축 파일 다운
files.download("/content/mosaic_rf.zip")


In [ ]:
# 파일 이름 수정용
import os

folder_path = "/content/drive/MyDrive/2nd_Project/Vision_Data/augmentation/crop_rotate270/images"

for filename in os.listdir(folder_path):
    if filename.startswith("rotate270_crop4"):
        new_filename = filename.replace("rotate270_crop4", "rotate270_crop1_")
        old_file_path = os.path.join(folder_path, filename)
        new_file_path = os.path.join(folder_path, new_filename)
        os.rename(old_file_path, new_file_path)


In [ ]:
# 제대로 이미지와 바운딩 박스 좌표가 맞는지 확인하는 코드

import os
import cv2
from google.colab.patches import cv2_imshow

CLASSES = ['bdot', 'bline1', 'bline2', 'bline3', 'bline4', 'bline5', 'bline6', 'bline7', 'bline8', 'break', 'circle', 'scratch', 'sdot', 'stain']
# class_id = int(label[0])


# 경로 설정
# image_folder_path = os.path.join(dataset.location, 'augmentations/images')
# label_folder_path = os.path.join(dataset.location, 'augmentations/labels')
image_folder_path = '/content/drive/MyDrive/2nd_Project/Vision_Data/train_accumulation1/images/hflip_mosaic_pattern_2-1-1-70-D_jpg.rf.2ddb418cc4fd1f27c5b87568e5a5abeb.png'
label_folder_path = '/content/drive/MyDrive/2nd_Project/Vision_Data/train_accumulation1/labels/hflip_mosaic_pattern_2-1-1-70-D_jpg.rf.2ddb418cc4fd1f27c5b87568e5a5abeb.txt'

# 이미지 파일과 라벨 파일을 매치해서 리스트로 저장
image_files = [f for f in os.listdir(image_folder_path) if os.path.isfile(os.path.join(image_folder_path, f))]
label_files = [f for f in os.listdir(label_folder_path) if os.path.isfile(os.path.join(label_folder_path, f))]

# 이미지 파일과 라벨 파일을 서로 매치해서 처리
for img_file in image_files:
    img_path = os.path.join(image_folder_path, img_file)
    label_file = img_file.rsplit('.', 1)[0] + ".txt"
    label_path = os.path.join(label_folder_path, label_file)

    # 이미지 로드
    image = cv2.imread(img_path)
    img_height, img_width = image.shape[:2]

    # 라벨 파일에서 바운딩 박스 정보 읽기
    with open(label_path, 'r') as label_file:
        for line in label_file:
            label = line.strip().split()
            class_id = int(label[0])
            x, y, w, h = map(float, label[1:])

            # 바운딩 박스 좌표를 절대 좌표로 변환
            x1 = int((x - w / 2) * img_width)
            y1 = int((y - h / 2) * img_height)
            x2 = int((x + w / 2) * img_width)
            y2 = int((y + h / 2) * img_height)

            # 바운딩 박스 그리기
            class_name = CLASSES[class_id]
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(image, class_name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    cv2_imshow(image)
    cv2.waitKey(0)

cv2.destroyAllWindows()


In [ ]:
# 증식 이미지에 바운딩 박스 한 장만 확인하고 싶을 때
import os
import cv2
from google.colab.patches import cv2_imshow

CLASSES = ['broken_line', 'circle', 'diagonal_line', 'dot', 'out_line', 'quarter_circle', 'scratch', 'semicircle', 'short_line', 'square_line', 'thick_line', 'thin_line']

# 특정 이미지 파일과 라벨 파일 경로 지정
img_file = 'hflip_mosaic_pattern_2-1-1-70-D_jpg.rf.2ddb418cc4fd1f27c5b87568e5a5abeb.png'
label_file = 'hflip_mosaic_pattern_2-1-1-70-D_jpg.rf.2ddb418cc4fd1f27c5b87568e5a5abeb.txt'
image_folder_path = '/content/drive/MyDrive/2nd_Project/Vision_Data/train_accumulation1/images'
label_folder_path = '/content/drive/MyDrive/2nd_Project/Vision_Data/train_accumulation1/labels'

# 이미지 파일과 라벨 파일 경로 생성
img_path = os.path.join(image_folder_path, img_file)
label_path = os.path.join(label_folder_path, label_file)

# 이미지 로드
image = cv2.imread(img_path)
img_height, img_width = image.shape[:2]

# 라벨 파일에서 바운딩 박스 정보 읽기
with open(label_path, 'r') as label_file:
    for line in label_file:
        label = line.strip().split()
        class_id = int(label[0])
        x, y, w, h = map(float, label[1:])

        # 바운딩 박스 좌표를 절대 좌표로 변환
        x1 = int((x - w / 2) * img_width)
        y1 = int((y - h / 2) * img_height)
        x2 = int((x + w / 2) * img_width)
        y2 = int((y + h / 2) * img_height)

        # 바운딩 박스 그리기
        class_name = CLASSES[class_id]
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image, class_name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

cv2_imshow(image)
cv2.waitKey(0)
cv2.destroyAllWindows()


# MODEL

In [ ]:
import ultralytics
ultralytics.checks()

import matplotlib.pyplot as plt
import torch
# import cv2
print(torch.__version__)


In [ ]:
!cat /content/drive/MyDrive/integration_class/data.yaml

In [ ]:
import yaml
import os


# # YOLOv8 학습과 검증에 사용되는 train, valid data가 저장되어 있는 디렉토리 경로
# # Detection하고 싶은 클래스 개수(7개)와 클래스에 대응되는 클래스 이름(names)
# data = { 'train' : os.path.join(dataset.location, "train"),
#          'val' : os.path.join(dataset.location, "valid"),
#          'test' : os.path.join(dataset.location, "test"),
#          'names' : ['broken_line', 'circle', 'diagonal_line', 'dot', 'out_line', 'quarter_line', 'scratch',
#                     'semicircle', 'short_line', 'square_line', 'thick_line', 'thin_line'],
#          'nc' : 12 }


# # 데이터 경로와 클래스 정보를 저장하고 있는 딕셔너리 객체, data를 YOLOv8 학습에 필요한 data.yaml 저장
# with open(os.path.join(dataset.location, 'data.yaml'), 'w') as f:
#     yaml.dump(data, f)

# # data.yaml 읽어서 화면에 출력
# with open(os.path.join(dataset.location, 'data.yaml'), 'r') as f:
#     data_yaml = yaml.safe_load(f)
#     display(data_yaml)

data = { 'train' : '/content/drive/MyDrive/semoconductor_rfm/train',
         'val' : '/content/drive/MyDrive/integration_class/valid',
         'test' : '/content/drive/MyDrive/semoconductor_rfm/test',
        #  'names' : ['broken_line', 'circle', 'diagonal_line', 'dot', 'out_line', 'quarter_circle', 'scratch',
        #             'semicircle', 'short_line', 'square_line', 'thick_line', 'thin_line'],
        #  'nc' : 12 }
         'names' : ['line', 'scratch', 'sdot'],
         'nc' : 3 }

# 데이터 경로와 클래스 정보를 저장하고 있는 딕셔너리 객체, data를 YOLOv8 학습에 필요한 data.yaml 저장
with open('/content/drive/MyDrive/integration_class/data.yaml', 'w') as f:
    yaml.dump(data, f)

# data.yaml 읽어서 화면에 출력
with open('/content/drive/MyDrive/integration_class/data.yaml', 'r') as f:
    data_yaml = yaml.safe_load(f)
    display(data_yaml)

In [ ]:
# MS COCO Dataset으로 사전 학습되어 있는 yolo의 nano모델을 로드하는 것이 일반적임
# yolov8s, yolov8m, yolov8l, yolov8x (뒤로 갈수록 무겁지만 정확도는 좋아짐)
from ultralytics import YOLO

# Load a model
# model = YOLO('yolov8n.yaml')  # build a new model from scratch (yolov8n.yaml파일로 새로 모델을 구축하겠다는 의미)
model = YOLO('yolov8n.pt')    # load a pretrained model (recommended for training)
# model = YOLO('/content/drive/MyDrive/2nd Project/YOLO V8/weights/weights-nano, robo_augmentations/last.pt')

In [ ]:
model.model.summary()

In [ ]:
model.model

In [ ]:
# 텐서 보드
# 포트 충돌 등 일어나면 - '!kill 107647' # pid임
%load_ext tensorboard
%tensorboard --logdir /content/runs/  # replace with 'runs' directory

In [ ]:
# custom data로 학습(파인튜닝)된 모델, 즉 best.pt, last.pt가 rund/detect/train에 저장되었다는 정보를 볼 수 있음
# default - epochs=100, patience=50, batch=16, imgsz=640, save=True,
# pretrained=True - 기본값임, 대규모 데이터셋으로부터 사전 학습된 가중치 사용하기에 좋음, 더욱이 우리가 가진 데이터셋의 수량은 적음
# amp = True - 반정밀도(16-bit)의 부동 소수점 연산을 사용하여 학습 속도 개선, 성능 차이는 별로 없음
# val = True

# <적용해 볼 것>
# optimizer = auto - 기본값임, Adam을 요즘은 많이 씀
# cos_lr = True - 학습률을 처음에는 크게, 갈수록 작게 조정

# Use the model
model.train(data=dataset.location+'/data.yaml', epochs=100)   # box=7.5, resume=True
# model.train(data='/content/drive/MyDrive/2nd Project/Vision Data/robo_dataset/yolov8-txt/data.yaml', epochs=100, patience=30, batch=32, imgsz=416)    # content앞에 / 있어야 함

In [ ]:
metrics = model.val(plots=True)

In [ ]:
# 모든 or 다양한 평가 수치 확인용
# metrics.box.map
# metrics.box.map50
# metrics.box.map75
# metrics.box.maps

In [ ]:
# 예측하고 싶은 테스트 데이터가 저장되어 있는 디렉토리
# runs/detect/predict 폴더에 test set에 대한 추론 결과 이미지 저장됨

# results = model.predict(source='/content/Aquarium_Data/test/images/', save=True)
results = model(dataset.location+'/test/images', save=True)  # predict on an image

In [ ]:
# export 하기
success = model.export(format='onnx')  # export the model to ONNX format
success


In [ ]:
# 코랩에 저장된 model을 구글 드라이브에 저장하여 보관
import os
import shutil

source_folder = os.path.join('/content/runs/detect/train', 'weights')
destination_folder = '/content/drive/MyDrive/2nd Project/YOLO V8/weights/'

# Ensure the destination folder exists, create it if not present
os.makedirs(destination_folder, exist_ok=True)

# Copy the contents of source_folder to the destination folder
shutil.copytree(source_folder, os.path.join(destination_folder, 'weights - 200'))


In [ ]:
# predicted images 저장
import os
import shutil

source_folder = os.path.join('/content/runs/detect/', 'predict')
destination_folder = '/content/drive/MyDrive/2nd Project/YOLO V8/predicted_images'

# Ensure the destination folder exists, create it if not present
os.makedirs(destination_folder, exist_ok=True)

# Copy the contents of source_folder_1 to the destination folder
shutil.copytree(source_folder, os.path.join(destination_folder, '18. predict - epoch 200'))


In [ ]:
# zip으로 로컬에 다운받기
import shutil
from google.colab import files

shutil.make_archive('/content/runs_archive', 'zip', '/content/runs')
files.download('/content/runs_archive.zip')


In [ ]:
img = '/content/runs/detect/predict/pattern_1-1-1-30-D_jpg.rf.796e6d8e544a43d362a684c175579425.jpg'
res = model(img)
res_plotted = res[0].plot()
# cv2_imshow(res_plotted)
plt.imshow(res_plotted[:,:,::-1])


In [ ]:
# 이미지 여러 장
import matplotlib.pyplot as plt
from PIL import Image
import requests
from io import BytesIO

image_urls= [
    'https://drive.google.com/uc?id=1K5fdYCwxHatDvlN6fjCQl1QKdLKgS-Fc',
    'https://drive.google.com/uc?id=1Qb0LkcFdqR2deHF4f3FOFlG2IP4eTvNx',
    'https://drive.google.com/uc?id=14ZdfTr7_JDa-ew0gs_tT7EIeAw3-0yCb',
    'https://drive.google.com/uc?id=1A4wL2SCxQ4c_N_9NkbfMEyQXdsZF6EdV'
]

titles = ['train_image', 'valid_image', 'test_image', 'predict_image']

# 이미지를 그리드 형태로 배열
fig, axes = plt.subplots(1, 4, figsize=(12, 4))

# 각 이미지를 가져와서 축(ax)에 표시
for i, ax in enumerate(axes):
    response = requests.get(image_urls[i])
    image = Image.open(BytesIO(response.content))
    ax.imshow(image)
    ax.axis('off')
    ax.set_title(titles[i])

# 이미지를 표시
plt.show()


In [ ]:
import os

file_list = os.listdir('/content/runs/detect/predict')

jpg_files = [file for file in file_list if file.endswith('.jpg')]

from PIL import Image

for jpg_file in jpg_files:
    image_path = os.path.join('/content/runs/detect/predict', jpg_file)
    image = Image.open(image_path)
    image.show()


In [ ]:
# 정규화하는 코드 - albumentations 써도 됨
# roboflow에서 상대적인 값으로 bounding box 좌표를 주므로, 신경 안쓰고 이미지 테이터만 정규화하면 됨

import os
from PIL import Image
import torch
import torchvision.transforms as transforms
import torchvision.utils as vutils

def normalize_image(image_path):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    image = Image.open(image_path)
    normalized_image = transform(image)
    return normalized_image

def normalize_images_in_folder(folder_path, save_folder):
    import torchvision  # Import torchvision inside the function

    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_path = os.path.join(root, file)
                normalized_image = normalize_image(image_path)

                # Additional step: Save the normalized image with a new filename
                save_filename = file
                save_path = os.path.join(save_folder, save_filename)
                torchvision.utils.save_image(normalized_image, save_path)

# 이미지들이 들어있는 폴더들에 대해 정규화 수행 및 저장
train_folder = "/content/drive/MyDrive/2nd Project/Vision Data/robo_dataset/yolov8-txt/train_ori/images"
valid_folder = "/content/drive/MyDrive/2nd Project/Vision Data/robo_dataset/yolov8-txt/valid_ori/images"
test_folder = "/content/drive/MyDrive/2nd Project/Vision Data/robo_dataset/yolov8-txt/test_ori/images"

normalized_train_save_folder = "/content/drive/MyDrive/2nd Project/Vision Data/robo_dataset/yolov8-txt/train/images"
normalized_valid_save_folder = "/content/drive/MyDrive/2nd Project/Vision Data/robo_dataset/yolov8-txt/valid/images"
normalized_test_save_folder = "/content/drive/MyDrive/2nd Project/Vision Data/robo_dataset/yolov8-txt/test/images"

# Then call the normalize_images_in_folder function as before
normalize_images_in_folder(train_folder, normalized_train_save_folder)
normalize_images_in_folder(valid_folder, normalized_valid_save_folder)
normalize_images_in_folder(test_folder, normalized_test_save_folder)
